# PuLP basics

FONTE: 

- https://www.youtube.com/watch?v=qNA7Y57bXLQ&list=PLr5INjIG1eXXdQuv5wZwreUyStGKsQIff

- https://scipbook.readthedocs.io/en/latest/flp.html#capacitated-facility-location-problem

**Instalando/carregando o módulo**

In [77]:
from pulp import *

**Definindo os conjuntos**

In [78]:
FOODS = ['OATMEAL', 'CHICKEN', 'EGGS', 'MILK']
CUSTUMER = [1, 2, 3, 4, 5]
FACILITY = ['FAC 1', 'FAC 2', 'FAC 3']

**Parâmetros**

In [79]:
dem = {1 : 80,
       2 : 270,
       3 : 250,
       4 : 160,
       5 : 170}

In [80]:
actcost = {'FAC 1' : 1000,
           'FAC 2' : 1000,
           'FAC 3' : 1000}

In [81]:
maxam = {'FAC 1' : 500,
         'FAC 2' : 500,
         'FAC 3' : 500,}

In [82]:
transp = {'FAC 1' : {1:4, 2:5, 3:6, 4:8, 5:10},
          'FAC 2' : {1:6, 2:4, 3:3, 4:5, 5:8},
          'FAC 3' : {1:9, 2:7, 3:4, 4:3, 5:4}}

**Definindo o problema**

In [83]:
prob = LpProblem("FacilityLocation", LpMinimize)

**Definindo as variáveis de decisão**

In [84]:
use_vars = LpVariable.dicts("UseLocation", FACILITY, 0, 1, LpBinary)

In [85]:
serv_vars = LpVariable.dicts("Service", [(i,j) for i in CUSTUMER
                                          for j in FACILITY],
                      0) #OBS: definindo uso de 'j' para FACILITY e 'i' para CUSTUMER

In [86]:
foods_vars = LpVariable.dicts("food", FOODS, 0)

**Função objetivo**

In [87]:
prob += lpSum(actcost[j]*use_vars[j] for j in FACILITY) \
        + lpSum(transp[j][i]*serv_vars[(i,j)] for j in FACILITY
                                              for i in CUSTUMER)

#OBS: usar atentamente os indices j e i.

**Constraints**

In [88]:
for i in CUSTUMER:
    prob += lpSum(serv_vars[(i,j)] for j in FACILITY) == dem[i]

In [89]:
for j in FACILITY:
    prob += lpSum(serv_vars[(i,j)] for i in CUSTUMER) <= maxam[j]

In [90]:
for i in CUSTUMER:
    for j in FACILITY:
        prob += serv_vars[(i,j)] <= dem[i]*use_vars[j]

**Solving**

In [91]:
prob.solve()

1

In [92]:
print('Status:', LpStatus[prob.status])

Status: Optimal


In [97]:
TOL = 0.000001
for i in FACILITY:
    if use_vars[i].varValue > TOL:
        print('Estabilish facility at site ', i, '.', sep='')

Estabilish facility at site FAC 2.
Estabilish facility at site FAC 3.


In [94]:
for v in prob.variables():
    print(v.name,'=',v.varValue)

Service_(1,_'FAC_1') = 0.0
Service_(1,_'FAC_2') = 80.0
Service_(1,_'FAC_3') = 0.0
Service_(2,_'FAC_1') = 0.0
Service_(2,_'FAC_2') = 270.0
Service_(2,_'FAC_3') = 0.0
Service_(3,_'FAC_1') = 0.0
Service_(3,_'FAC_2') = 150.0
Service_(3,_'FAC_3') = 100.0
Service_(4,_'FAC_1') = 0.0
Service_(4,_'FAC_2') = 0.0
Service_(4,_'FAC_3') = 160.0
Service_(5,_'FAC_1') = 0.0
Service_(5,_'FAC_2') = 0.0
Service_(5,_'FAC_3') = 170.0
UseLocation_FAC_1 = 0.0
UseLocation_FAC_2 = 1.0
UseLocation_FAC_3 = 1.0


In [95]:
print('Total cost of food/serving', value(prob.objective))

Total cost of food/serving 5570.0


***

In [4]:
from pulp import *
import pandas as pd

In [3]:
prob

Simple_diet_problem:
MINIMIZE
None
VARIABLES

In [24]:
file_path = '/home/anderson/Downloads/diet - medium.xls'

df = pd.read_excel(file_path, nrows=17)

In [25]:
df.tail()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
12,Apple Pie,0.48,1 Oz,67.2,0.0,3.1,75.4,9.6,0.5,0.5,35.2,0.9,3.1,0.1
13,Scrambled Eggs,0.33,1 Egg,99.6,211.2,7.3,168.0,1.3,0.0,6.7,409.2,0.1,42.6,0.7
14,Turkey Bologna,0.45,1 Oz,56.4,28.1,4.3,248.9,0.3,0.0,3.9,0.0,0.0,23.8,0.4
15,Beef Frankfurter,0.81,1 Frankfurter,141.8,27.4,12.8,461.7,0.8,0.0,5.4,0.0,10.8,9.0,0.6
16,Chocolate Chip Cookies,0.09,1 Cookie,78.1,5.1,4.5,57.8,9.3,0.0,0.9,101.8,0.0,6.2,0.4


In [153]:
#instantiation of LP object
prob = LpProblem('Simple diet problem', LpMinimize)

In [154]:
#sets
food_items =  df['Foods'].to_list()

In [155]:
#constants
costs = dict(zip(food_items, df['Price/Serving'].to_list()))
calories = dict(zip(food_items, df['Calories']))
fat = dict(zip(food_items,df['Total_Fat (g)']))
carbs = dict(zip(food_items,df['Carbohydrates (g)']))
proteins = dict(zip(food_items, df['Protein (g)']))

In [156]:
#variables
food_vars = LpVariable.dicts('Food', food_items, lowBound=0, cat='Continuous')

In [157]:
#OBJECTIVE FUNTION
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

In [158]:
#constraint: calories
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0

In [159]:
#constraint: fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0

In [160]:
#constraint: carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0

In [161]:
#constraint: carbs
prob += lpSum([proteins[f] * food_vars[f] for f in food_items]) >= 100.0
prob += lpSum([proteins[f] * food_vars[f] for f in food_items]) <= 150.0

In [162]:
#inspecting LP object
prob

Simple_diet_problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.44999999999999996*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.9299999999999999*Food_Tofu + 0.44999999999999996*Food_Turkey_Bologna + 0.15000000000000002*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
SUBJECT TO
_C1: 67.2 Food_Apple_Pie + 104.9 Food_Banana + 141.8 Food_Beef_Frankfurter
 + 78.1 Food_Chocolate_Chip_Cookies + 73.8 Food_Frozen_Broccoli
 + 72.2 Food_Frozen_Corn + 81 Food_Oatmeal_Cookies + 81.4 Food_Raw_Apple
 + 2.6 Food_Raw_Lettuce_Iceberg + 277.4 Food_Roasted_Chicken
 + 99.6 Food_Scrambled_Eggs + 358.2 Food_Spaghetti_W__Sauce + 88.2 Food_Tofu
 + 56.4 Food_Turkey_Bologna + 65 Food_Wheat_Bread + 65 Food_White_Bread
 + 171.5 Food__Baked_Potatoes

In [163]:
#solving LP
%time prob.solve()

CPU times: user 8.93 ms, sys: 4.35 ms, total: 13.3 ms
Wall time: 25.1 ms


1

In [164]:
#inspecting status of the solution
LpStatus[prob.status]

'Optimal'

In [165]:
#inspecting the values for the variables
{v : v.varValue for v in prob.variables()} 

{Food_Apple_Pie: 0.0,
 Food_Banana: 0.0,
 Food_Beef_Frankfurter: 0.0,
 Food_Chocolate_Chip_Cookies: 0.0,
 Food_Frozen_Broccoli: 6.9242113,
 Food_Frozen_Corn: 0.0,
 Food_Oatmeal_Cookies: 0.0,
 Food_Raw_Apple: 0.0,
 Food_Raw_Lettuce_Iceberg: 0.0,
 Food_Roasted_Chicken: 0.0,
 Food_Scrambled_Eggs: 6.060891,
 Food_Spaghetti_W__Sauce: 0.0,
 Food_Tofu: 0.0,
 Food_Turkey_Bologna: 0.0,
 Food_Wheat_Bread: 0.0,
 Food_White_Bread: 0.0,
 Food__Baked_Potatoes: 1.0806324}

In [166]:
#RESULT: found value for the diet cost
value(prob.objective)

5.518229286

**Same problem, but restricting variables as integers numbers (integer programming problem)**

In [167]:
#instantiation of LP object
prob = LpProblem('Simple diet problem - Integer', LpMinimize)

In [182]:
#sets
food_items =  df['Foods'].to_list()

In [169]:
#costants
costs = dict(zip(food_items, df['Price/Serving'].to_list()))
calories = dict(zip(food_items, df['Calories']))
fat = dict(zip(food_items,df['Total_Fat (g)']))
carbs = dict(zip(food_items,df['Carbohydrates (g)']))
proteins = dict(zip(food_items, df['Protein (g)']))

In [170]:
#variables
food_vars = LpVariable.dicts('Food', food_items, lowBound=0, cat='Integer') #HERE IS THE DIFFERENCE!! (compare with the previous code!!)

In [171]:
#OBJECTIVE FUNTION
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

In [172]:
#constraint: calories
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0

In [173]:
#constraint: fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0

In [174]:
#constraint: carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0

In [175]:
#constraint: carbs
prob += lpSum([proteins[f] * food_vars[f] for f in food_items]) >= 100.0
prob += lpSum([proteins[f] * food_vars[f] for f in food_items]) <= 150.0

In [176]:
#solving LP
%time prob.solve() #It will take longer time than before..

CPU times: user 6.26 ms, sys: 8.25 ms, total: 14.5 ms
Wall time: 32.1 ms


1

In [177]:
#inspecting status of the solution
LpStatus[prob.status]

'Optimal'

In [179]:
#inspecting the values for the variables
{v : v.varValue for v in prob.variables() if v.varValue > 0} 

{Food_Frozen_Broccoli: 7.0,
 Food_Raw_Lettuce_Iceberg: 1.0,
 Food_Scrambled_Eggs: 6.0,
 Food__Baked_Potatoes: 1.0}

In [180]:
#RESULT: found value for the diet cost
value(prob.objective)

5.58

***

In [237]:
prob = LpProblem('Brewery', LpMaximize)

In [238]:
#set
beer_items = ['Ale', 'Beer']

In [239]:
#constants
consts = dict( zip(beer_items, [13, 23]) )

rc1 = dict( zip(beer_items, [4, 4]) )
rc2 = dict( zip(beer_items, [35, 20]) )
rc3 = dict( zip(beer_items, [5, 15]) )

In [240]:
#variables
beer_vars = LpVariable.dicts('beer', beer_items, lowBound=0, cat='Integer')

In [241]:
#objective funtion
prob += lpSum([beer_vars[i]*consts[i] for i in beer_items] )

In [242]:
#constraints
prob += lpSum([rc1[i]*beer_vars[i] for i in beer_items]) <= 160
prob += lpSum([rc2[i]*beer_vars[i] for i in beer_items]) <= 1190
prob += lpSum([rc3[i]*beer_vars[i] for i in beer_items]) <= 480

In [243]:
#inspecting LP
prob

Brewery:
MAXIMIZE
13*beer_Ale + 23*beer_Beer + 0
SUBJECT TO
_C1: 4 beer_Ale + 4 beer_Beer <= 160

_C2: 35 beer_Ale + 20 beer_Beer <= 1190

_C3: 5 beer_Ale + 15 beer_Beer <= 480

VARIABLES
0 <= beer_Ale Integer
0 <= beer_Beer Integer

In [244]:
#solving...
prob.solve()

1

In [245]:
#output status
LpStatus[prob.status]

'Optimal'

In [246]:
#inspecting the values for the variables
{v : v.varValue for v in prob.variables() if v.varValue > 0} 

{beer_Ale: 12.0, beer_Beer: 28.0}

In [247]:
value(prob.objective)

800.0